# Обработка исключений в C++

Обработка исключений в C++ реализуется с помощью конструкции try-catch, throw.

try - область в которой может выполниться ошибка

catch - область в которой ошибка обрабатывается

throw - "вброс" ошибки

In [36]:
#include <iostream>

using namespace std;

```C++
int a, b;
cin >> a >> b;
int c = a / b;
cout << c << endl;
// 1 0 - исключительная ситуация
```

In [55]:
int a, b;
cin >> a >> b;
try {
    if (b == 0) throw "Zero division error";
    int c = a / b;
    cout << "c = " << c << endl;
    if (a == 0) throw -1;
}
catch (const char *error) {
    cout << error << endl;
}
catch (...) {
    // работает на любой throw, который не попал в предыдущие catch
    cout << "Exception!" << endl;
}

a b
Zero division error


Если throw использовать вне блока try-catch, то программа завершит свою работу, будет указана причина завершения процесса.

```cpp
void throw_func() {
    throw 0;
}

throw_func();
```

Вывод:

$\color{darkred}{\text{terminate called after throwing an instance of 'int'}}$

## Пример реализации квадратного уравнения на C++

In [38]:
#include <cmath>

In [39]:
const int MAX_LENGTH = 100;

In [40]:
bool is_null(double x) {
    const double delta = 1e-7;
    return abs(x) <= delta;
}

In [41]:
struct QuadraticEquationError {
    virtual void print() const = 0;
};

In [42]:
struct LinearEquationException : QuadraticEquationError {
    double x;

    LinearEquationException(double b, double c) : x(-c / b) {}

    void print() const override {
        printf("x = %f\n", x);
    }
};

In [43]:
struct DiscriminatorEqualsZeroException : QuadraticEquationError {
    double x;

    DiscriminatorEqualsZeroException(double a, double b) : x(-b / (2 * a)) {}

    void print() const override {
        printf("x1 = x2 = %f\n", x);
    }
};

In [44]:
struct DiscriminatorLowerThatZeroException : QuadraticEquationError {
    double real, img;

    DiscriminatorLowerThatZeroException(double D, double a, double b) {
        real = -b / (2 * a);
        img = sqrt(abs(D)) / (2 * a);
    }

    void print() const override {
        if (is_null(real)) {
            printf("x1 = %fi\nx2 = -%fi\n", img, img);
            return;
        }
        printf("x1 = %f + %fi\n", real, img);
        printf("x1 = %f - %fi\n", real, img);
    }
};

In [45]:
void read_char(char *ch) {
    char tmp;
    int i = 0;
    cin.get(tmp);
    while (tmp != ' ' and tmp != '\n' and tmp != '\0') {
        ch[i] = tmp;
        i++;
        if (i >= MAX_LENGTH) throw -1;
        cin.get(tmp);
    }
    cin.clear();
}

In [46]:
double get_double(const char *input) {
    bool point_state = false;
    bool sign_state = false;
    int i;
    for (i = 0; i < MAX_LENGTH; i++) {
        char tmp = input[i];
        if (tmp == '\0') break;
        if (tmp == '+' or tmp == '-') {
            if (sign_state) throw 1;
            sign_state = true;
        } else if (tmp == '.') {
            if (point_state) throw 2;
            point_state = true;
        } else if (tmp < '0' or tmp > '9') throw 3;
    }
    return atof(input);
}

In [47]:
void solve(char *_a, char *_b, char *_c) {
    try {
        double a = get_double(_a);
        double b = get_double(_b);
        double c = get_double(_c);

        if (is_null(a) and is_null(b) and is_null(c)) throw 4;
        if (is_null(a) and is_null(b)) throw 5;
        if (is_null(a)) throw LinearEquationException(b, c);

        double D = b * b - 4 * a * c;
        if (D == 0) throw DiscriminatorEqualsZeroException(a, b);
        if (D <= 0) throw DiscriminatorLowerThatZeroException(D, a, b);
        double x1 = (-b + sqrt(D)) / (2 * a);
        double x2 = (-b - sqrt(D)) / (2 * a);
        printf("x1 = %f\n", x1);
        printf("x2 = %f\n", x2);
    }
    catch (const int error) {
        if (error == 1) cout << "Sign doubles in input!" << endl;
        else if (error == 2) cout << "Point doubles in input!" << endl;
        else if (error == 3) cout << "Not a number in input!" << endl;
        else if (error == 4) cout << "x = R" << endl;
        else if (error == 5) cout << "Solution doesn't exist!" << endl;
    }
    catch (const QuadraticEquationError &error) {
        error.print();
    }
    catch (...) {
        cout << "A error has occurred!" << endl;
    }
}

In [48]:
void solve_qe() {
    char *a = new char[MAX_LENGTH];
    char *b = new char[MAX_LENGTH];
    char *c = new char[MAX_LENGTH];
    try {
        read_char(a);
        read_char(b);
        read_char(c);
    }
    catch (int error) {
        if (error == -1) cout << "Too long number!" << endl;
        delete[] a, b, c;
        return;
    }
    catch (...) {
        delete[] a, b, c;
        return;
    }
    solve(a, b, c);
    delete[] a, b, c;
}